In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from groq import Groq
from langchain_community.document_loaders import WikipediaLoader
from langchain.schema import Document
from datetime import datetime,timedelta


In [3]:
client = Groq(api_key = os.getenv("GROQ_API_KEY"))

In [4]:
AVAILABLE_MODELS = [
    "allam-2-7b",
    "compound-beta",
    "compound-beta-mini",
    "deepseek-r1-distill-llama-70b",
    "gemma2-9b-it",
    "llama-3.1-8b-instant",
    "llama-3.3-70b-versatile",
    "meta-llama/llama-4-maverick-17b-128e-instruct",
    "meta-llama/llama-4-scout-17b-16e-instruct",
    "meta-llama/llama-guard-4-12b",
    "meta-llama/llama-prompt-guard-2-22m",
    "meta-llama/llama-prompt-guard-2-86m",
    "moonshotai/kimi-k2-instruct",
    "openai/gpt-oss-120b",
    "openai/gpt-oss-20b",
    "qwen/qwen3-32b"
]


In [5]:
def create_knowledge_base():
    topics = ["Machine Learning","Deep Learning","Data Science","Explainable Ai","Agentic AI"]
    documents = []
    
    for topic in topics:
        try:
            loader = WikipediaLoader(query = topic,load_max_docs = 1)
            docs = loader.load()
            documents.extend(docs)
            print(f"Loaded Content For : {topic}")
        except Exception as e:
            print(f"could not load content for {topic } : {e}")
            fallback_content = f"{topic} is a fascinating subject that involves systematic content Tell no data available for the given topic "
            documents.append(Document(page_content = fallback_content))
            
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
    knowledge_base = Chroma.from_documents(texts,embeddings)
    print("Knowledge base Created successfully")
    return knowledge_base    
knowledge_base = create_knowledge_base()


Loaded Content For : Machine Learning
Loaded Content For : Deep Learning
Loaded Content For : Data Science
Loaded Content For : Explainable Ai
Loaded Content For : Agentic AI


C:\Users\ravik\AppData\Local\Temp\ipykernel_22200\1206283098.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


Knowledge base Created successfully


In [6]:
def retrive_educational_content(query  :str)-> str:
    try : 
        docs = knowledge_base.similarity_search(query,k = 3)
        content = "\n\n".join([f"Source {i+1} :\n {doc.page_content[:500]}..." for i,doc in enumerate(docs)])
        return content
    except Exception as e:
        return f"Information about {query} : Error retrieving content - {str(e)}"
    


In [7]:

def generate_quiz(topic: str, difficulty: str = "Beginner", model: str = "deepseek-r1-distill-llama-70b") -> str:
  """Generate a quiz on a specific topic."""
  prompt = f"""Create a {difficulty} level quiz about {topic} with 3 questions.
  Format i clearly with:
  1. Question 1
  2. Question 2
  3. Question 3
  Then provide answers at the end.
  """
  
  try:
      chat_completion = client.chat.completions.create(
          messages = [{"role":"user","content" : prompt}],
          model = model,
          max_tokens = 500)
      return chat_completion.choices[0].message.content
  except Exception as e:
      return f"could not generate quiz : {str(e)}"


In [8]:
def schedule_session(topic  :str,student_level : str,model : str)->str:
    prompt = f"""
    As an AI scheduling assistant, created a personlized study for learning {topic} at
    {student_level} level . 
    Include:
    -Recommended session times
    - Duration for each session
    -topic to cover in each session
    - Breaks and review periods
    """
    
    try: 
        chat_completion = client.chat.completions.create(
            messages = [{"role" : "user","content" : prompt}],
            model = model,
            max_tokens = 300
        )
        schedule_content = chat_completion.choices[0].message.content
        session_time = (datetime.now() + timedelta(hours = 24)).strftime("%Y-%m-%d at %H:%M" )
        return f"{schedule_content}\n\nFirst session scheduled on '{topic}' for {student_level} level at {session_time} .Remainder set!"
    except Exception as e:
        session_time = (datetime.now() + timedelta(hours=24)).strftime("%Y-%m-%d at %H:%M")
        return f"scheduled session on '{topic}' for {student_level} level at {session_time}.Remainder set!"
    

In [9]:
def explain_concept(concept : str , student_level :str , model : str = "deepseek-r1-distill-llama-70b") -> str:
    prompt = f""" 
    Explain a comncept inm simple terms with examples.
    Explain {concept} in simple terms for a  {student_level} level student  .
    Use clear language, practical example , and keep it under 300 words.
    structure you explanation with :
    - Basic definition
    - key concepts
    - Real-world exapmles 
    - Why it's important
    
    """
    try : 
        chat_completion = client.chat.completions.create(
            messages = [{"role" : "user" , "content" : prompt}],
            model = model,
            max_tokens = 500
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Could not generate a explanation : {str(e)}"
    

In [10]:
def provide_feedback(topic : str ,student_responses : str ,  student : str ,student_level : str ,model : str = "deepseek-r1-distill-llama-70b") ->str :
    if not student_responses.strip():
        return "No student responses provided for feedback"
    prompt = f""" Provide constructive feedback on these { student_level} level student responses about {topic} : 
    
    student Answers :
    {student_responses}
    
    provide:
    1. Positive renforcement
    2.specific areas for imporovemet
    3.Suggestions for better understanding
    4.Encouragement for continued learning
    """
    
    try:
        chat_completion = client.chat.completions.create(
            messages = [{"role" : "user" , "content"  :prompt}],
            model = model,
            max_tokes = 300
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        return f"could not generate  feedback : {str(e)}"
        
        

In [11]:
def create_learning_path(topic: str, student_level: str, difficulty: str, model: str = "deepseek-r1-distill-llama-70b") -> str:

  """Create a personalized learning path for the student."""
  prompt = f"""
  Create a step by learning path 
  Include : 
  - Learnig objectives
  - Recommend study
  - key milestones
  - suggested resources
  - practice excercises
  """ 

  try:
    chat_completion = client.chat.completions.create(
        messages =[{"role":"user", "content":prompt}],
        model = model,
        max_tokens = 400
    )
    return chat_completion.choices[0].message.content

  except Exception as e:
    return f"Could not generate learning path: {str(e)}"

In [13]:
# Simplified tutoring function
def simplified_tutoring(topic , student_level , difficulty , student_responses , model):
  """Main tutoring function that coordinates all agents"""
  results = []
  results.append("Multi-Agent Tutoring System Initialized * ")
  results.append(f"Topic: {topic}")
  results.append(f"level: * {student_level}")
  results.append(f"Difficulty: * {difficulty}")
  results.append(f"* Ai Model * {model}")
  results.append("="*60)
    
  results.append("\n *Lead TUTOTR : Creating Learning Path *")
  learning_path = create_learning_path(topic,student_level,difficulty,model)
  results.append(learning_path)

  results.append("\n*Content RETRIEVER: findind respurces")
  content = retrive_educational_content(topic)
  results.append(content)

  results.append("\n*CONCEPT EXPLAINER:Clear Explanation*")
  explanation=explain_concept(topic, student_level,model)
  results.append(explanation)

  results.append("\n*QUIZ GENERATOR:Generating Quiz*")
  quiz = generate_quiz(topic,difficulty,model)
  results.append(quiz)

  results.append("\n*SESSION SCHEDULER:Scheduling Session*")
  session_schedule = schedule_session(topic,student_level,model)
  results.append(session_schedule)

  results.append("\n*FEEDBACK GENERATOR:Provide Feedback*")
  feedback = provide_feedback(topic,student_responses,student_level,model)
  results.append(feedback)

  results.append("\n"+"="*60)
  results.append("Tutoring session completed")
  return "\n".join(results)

In [18]:
import gradio as gr
def tutoring_interface(topic, student_level, difficulty, student_responses, model):
    try:
        # Validate inputs
        if not topic.strip():
            return "Please enter a topic to learn about!"


        print(f"Starting tutoring session: {topic}, {student_level}, {difficulty}, {model}")
        result = simplified_tutoring(topic, student_level, difficulty, student_responses, model)
        return result


    except Exception as e:
        error_msg = f"Error in tutoring system: {str(e)}"
        error_msg += "\n\n Troubleshooting tips:"
        error_msg += "\n1. Check your Groq API key is valid and set"
        error_msg += "\n2. Ensure you have internet connection"
        error_msg += "\n3. Try a different topic or simpler query"
        error_msg += "\n4. Some models might not be available in your region"
        return error_msg


# Set up the Gradio interface
with gr.Blocks(title="AI Tutoring System", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    #  Multi-Agent AI Tutoring System
    Get personalized tutoring across various subjects with our AI-powered multi-agent system
    """)


    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("###  Learning Preferences")
            topic = gr.Textbox(
                label="What would you like to learn about?",
                placeholder="E.g., Quantum Physics, Calculus, Machine Learning...",
                value="Quantum Mechanics"
            )
            student_level = gr.Dropdown(
                ["Beginner", "Intermediate", "Advanced"],
                label="Your current level",
                value="Beginner"
            )
            difficulty = gr.Dropdown(
                ["Easy", "Medium", "Hard"],
                label="Desired difficulty level",
                value="Medium"
            )
            model = gr.Dropdown(
                AVAILABLE_MODELS,
                label="Select AI Model",
                value="deepseek-r1-distill-llama-70b",
                info="Choose which model to use for content generation"
            )
            student_responses = gr.Textbox(
                label="Your previous answers (for feedback)",
                lines=3,
                value="",
                placeholder="Paste your quiz answers here for personalized feedback..."
            )
            submit_btn = gr.Button(" Start Learning Session", variant="primary")


            gr.Markdown("###  Tips")
            gr.Markdown("""
            - Be specific with your topic
            - Choose the level that matches your knowledge
            - Use the feedback section to get personalized tips
            - Different models may provide different response styles
            - Larger models (like 70B) generally provide more detailed responses
            """)


        with gr.Column(scale=2):
            gr.Markdown("###  Tutoring Session Output")
            output = gr.Textbox(
                label="Session Results",
                lines=25,
                interactive=False,
                show_copy_button=True
            )


    # Examples
    gr.Markdown("###  Example Learning Topics")
    examples = gr.Examples(
        examples=[
            ["Quantum Physics", "Beginner", "Easy", "", "deepseek-r1-distill-llama-70b"],
            ["Machine Learning", "Intermediate", "Medium", "", "llama-3.3-70b-versatile"],
            ["Calculus", "Intermediate", "Medium", "", "gemma2-9b-it"],
            ["Python Programming", "Beginner", "Easy", "", "llama-3.1-8b-instant"],
            ["Neural Networks", "Advanced", "Hard", "", "qwen/qwen3-32b"]
        ],
        inputs=[topic, student_level, difficulty, student_responses, model],
        label="Click any example to load it"
    )


    # Connect the button
    submit_btn.click(
        fn=tutoring_interface,
        inputs=[topic, student_level, difficulty, student_responses, model],
        outputs=output
    )


# Launch the application
if __name__== "__main__":
    print("Starting AI Tutoring System...")
    print("Available models:", AVAILABLE_MODELS)


    # Test the API key
    try:
        test_response = client.chat.completions.create(
            messages=[{"role": "user", "content": "Hello"}],
            model="deepseek-r1-distill-llama-70b",
            max_tokens=10
        )
        print("Groq API connection successful!")
    except Exception as e:
        print(f"Groq API error: {e}")
        print("Please check your API key and try again.")


    demo.launch(share=True, debug=True)

Starting AI Tutoring System...
Available models: ['allam-2-7b', 'compound-beta', 'compound-beta-mini', 'deepseek-r1-distill-llama-70b', 'gemma2-9b-it', 'llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'meta-llama/llama-4-scout-17b-16e-instruct', 'meta-llama/llama-guard-4-12b', 'meta-llama/llama-prompt-guard-2-22m', 'meta-llama/llama-prompt-guard-2-86m', 'moonshotai/kimi-k2-instruct', 'openai/gpt-oss-120b', 'openai/gpt-oss-20b', 'qwen/qwen3-32b']
Groq API connection successful!
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/09/03 12:25:29 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


Starting tutoring session: Machine Learning, Intermediate, Hard, openai/gpt-oss-20b
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None
